In [ ]:
from __future__ import division

In [ ]:
import sys
sys.path.append('..')

In [ ]:
import math
import random
import numpy as np
import pandas as pd

In [4]:
from sklearn import datasets

In [5]:
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
%load_ext autoreload
%autoreload 1

%aimport gtree
%aimport tools

In [7]:
import logging
%config Application.log_level="INFO"
logging.basicConfig(format='%(asctime)s %(name)s %(levelname)s:%(message)s', level=logging.DEBUG, datefmt='%I:%M:%S')

## Goals:

### Separate the structure of a tree from the data of a tree.  In other words,
  fitting a tree does two things: It creates the structure of a tree and it
  creates a mapping of each leaf to a value.  Lookup therefore requires both
  finding the leaf node AND using the map to lookup the value.
  
### The loss function optimized by the tree is configurable, as is the leaf


## Terms:

### Tree
A Tree is an object that takes input data and determines what leaf it ends up in.  Unlike many tree implementations, the Tree itself doesn't store data about the value of a leaf.  That is stored externally.


### loss_fn
A loss_fn is a function that takes data rows, the predicted targets for those rows, and the actual targets for those rows, and returns a single value that determines the "LOSS" or "COST" of that prediction (lower cost/loss is better)

```
def loss_fn(predicted_targets, actual_targets) -> float
```

A loss function must be additive (so, one should not apply a mean as a part of it)

### leaf_prediction_fn
A leaf_prediction_fn is a function which takes the features and actual targets that end up in a leaf and returns a Series of the predictions for each row ending up in that leaf.  It is typically a constant function whose value is either the mean good rate in that leaf (among the actual targets) or the median target, but can be anything else

```
def leaf_prediction_fn(features) -> pd.Series
```


### leaf_prediction_builder
A leaf_prediction_builder is a function which takes the features and actual targets that end up in a TRANING leaf and returns a leaf_prediction_fn.  This leaf_prediction_fn is used to predict the value of testing rows that end up in the same leaf.

```
def leaf_prediction_builder(features, actual_targets) -> leaf_prediction_fn
```


### leaf_prediction_map
A leaf_prediction_map is a map of leaf ids (eg their hash) to the leaf_prediction_fn for that leaf.  One can only use a tree to score data if one has a leaf_prediction_map.  This design allows on to use the same tree as a subset of another tree without having their leaf values become entangled.

--------------

## Test Tree Manipulation Functions

In [ ]:
%pdb off

In [ ]:
t = gtree.BranchNode('A', 0.5, None, None)
t.left = gtree.LeafNode()
t.right = gtree.BranchNode('B', 0.9, None, None)
t.right.left = gtree.LeafNode()
t.right.right = gtree.LeafNode()

o = gtree.BranchNode('C', 0.1, None, None)
o.left = gtree.LeafNode()
o.right = gtree.LeafNode()

t.prn()
print '\n\n'
o.prn()

In [ ]:
u = gtree.replace_branch_split(t, t.right, o)
u.prn()
print '\n\n'
t.prn()

In [ ]:
v = gtree.replace_node(t, t.left, o)
v.prn()
print '\n\n'
t.prn()

In [ ]:
t = gtree.BranchNode('A', 0.5, None, None)
t.left = gtree.LeafNode()
t.right = gtree.BranchNode('B', 0.9, None, None)
t.right.left = gtree.LeafNode()
t.right.right = gtree.BranchNode('C', 0.9, None, None)
t.right.right.right = gtree.LeafNode()
t.right.right.left = gtree.LeafNode()

t.prn()
print '\n\n'
gtree.prune(t, 2).prn()
print '\n\n'
t.prn()

In [ ]:
data = pd.DataFrame({'A': [0.1, 10, .02],
                     'B': [10, 20, 30]},
                    index=['foo', 'bar', 'baz'])

In [ ]:
class StaticLeaf(object):

    def __init__(self, val):
        self.val = val
        
    def predict(self, df):
        return np.array([self.val for _ in range(len(df))])

In [ ]:
t = gtree.BranchNode('A', 0.5, None, None)
t.left = gtree.LeafNode() #'A', 0.5, 10, 20)
t.right = gtree.LeafNode() #'A', 0.5, 100, 0)

leaf_map = {hash(t.left): StaticLeaf(10),
            hash(t.right): StaticLeaf(20)}

t.predict(data, leaf_map)

In [ ]:
t

In [ ]:
# Create a split on a DataFrame

df = pd.DataFrame({'foo': pd.Series([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])})

gtree._single_variable_best_split(
    df,
    'foo',
    pd.Series([0, 0, 1, 0, 0, 1, 1, 0, 1, 1]),
    loss='error_rate',
    leaf_prediction='mean')

In [ ]:
threshold = 0.5
truth     = pd.Series([1, 0, 1], dtype=np.float32)
predicted = pd.Series([0, 1, 1], dtype=np.float32)

print gtree.loss(truth, predicted, type='error_rate')

print 1.0 - ((predicted >= threshold) == truth).mean() #+ (predicted < threshold) * (1 - truth)

# Test Split Finding

In [ ]:
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                   'B': [10, 20, 50, 30, 40, 50, 60, 50, 70, 90, 100, 110 ]}, dtype=np.float32)
target = pd.Series([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0], dtype=np.float32)

tree, leaf_map = gtree.train_greedy_tree(df, target, loss='error_rate')

print '\nTree:\n'
tree.prn()

print leaf_map

In [ ]:
gtree.calculate_leaf_map(tree, df, target)

In [ ]:
gtree.random_node(tree)

In [ ]:
print gtree.get_all_nodes(tree)

In [ ]:
df = pd.DataFrame({'A': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
                   'B': [10, 20, 50, 30, 40, 50, 60, 50, 70, 90, 100, 110 ]}, dtype=np.float32)
target = pd.Series([0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0], dtype=np.float32)

In [ ]:
gtree._single_variable_best_split(df,'B', target,
                                  loss='error_rate',
                                  leaf_prediction='mean')

In [ ]:
tree, leaf_map = gtree.train_greedy_tree(df, target,
                                         loss='error_rate',
                                         feature_sample_rate=.5,
                                         row_sample_rate=.5)

print '\nTree:\n'
tree.prn()

print leaf_map

In [ ]:
gtree.mate(tree, tree).prn()

In [ ]:
#def make_hastie_sample(n_samples):
#
#    features, targets = datasets.make_hastie_10_2(n_samples=n_samples)
#
#    features = pd.DataFrame(features, columns=['feature_{}'.format(i) for i in range(features.shape[1])])
#    targets = pd.Series(targets, name='target')
#    targets = targets.map(lambda x: 1.0 if x > 0 else 0.0)
#    return features, targets



In [ ]:
#def make_kddcup(n_samples):
#    
#    features, targets = datasets.fetch_kddcup99(subset='smtp')
#
#    features = pd.DataFrame(features, columns=['feature_{}'.format(i) for i in range(features.shape[1])])
#    targets = pd.Series(targets, name='target')
#    targets = targets.map(lambda x: 1.0 if x > 0 else 0.0)
#    
#    features = featurse.sample(n=n_samples)
#    
#    return features, targets.loc[features.index]
    

#def make_random_classification(n_samples, n_features=100):
       
#    features, targets = datasets.make_classification(n_samples=n_samples,
#                                                     n_features=n_features,
#                                                     n_informative=8,
#                                                     n_classes=2,
#                                                     n_clusters_per_class=4)
#                                                     
#    features = pd.DataFrame(features, columns=['feature_{}'.format(i) for i in range(features.shape[1])])
#    targets = pd.Series(targets, name='target')
#    targets = targets.map(lambda x: 1.0 if x > 0 else 0.0)
#        
#    return features, targets.loc[features.index]

# Start the Test Analysis Here

In [ ]:
#features, targets = make_hastie_sample(10000)
features, targets = tools.make_random_classification(10000)
features = pd.DataFrame(features, dtype=np.float32)
targets = pd.Series(targets, dtype=np.float32)

In [ ]:
features.shape

In [ ]:
targets.value_counts()

In [ ]:
gtree.tree_logger.setLevel(logging.INFO)
tree, leaf_map = gtree.train_greedy_tree(features, targets,
                                         loss='cross_entropy',
                                         leaf_prediction='mean',
                                         max_depth=7)

In [ ]:
#gtree.cross_entropy_loss(targets[features.feature_45 < .2326])

In [ ]:
#gtree._single_variable_best_split(features, 'feature_45', targets, None, None, None)

In [ ]:
tree.prn()

In [ ]:
set(pd.Series([1, 2, 3]))

In [ ]:
tree.predict(features, leaf_map)

In [ ]:
results = pd.DataFrame({'truth': targets, 'prediction': tree.predict(features, leaf_map)})

In [ ]:
1.0 - gtree.error_rate_loss(results.prediction, results.truth) / len(targets)

In [ ]:
results.plot(kind='scatter', x='prediction', y='truth')

In [ ]:
fig = plt.figure(figsize=(12,8))

for label, grp in tree.predict(features, leaf_map).groupby(targets):
    grp.hist(normed=True, alpha=0.5, label=str(label)) #, label=label)
plt.legend(loc='best')
None

# Compare Methods

In [ ]:
features, targets = tools.make_random_classification(5000)
features = pd.DataFrame(features, dtype=np.float32)
targets = pd.Series(targets, dtype=np.float32)

features_validation = features.sample(frac=.3)
targets_validation = targets.loc[features_validation.index]

features = features[~features.index.isin(features_validation.index)]
targets = targets.loc[features.index]

In [ ]:
%pdb off

In [ ]:
gtree.tree_logger.setLevel(logging.WARNING)

result, generations = gtree.evolve(features, targets,
                                   loss='cross_entropy',
                                   max_depth=3, min_to_split=10,
                                   num_generations=15, num_survivors=10,
                                   num_children=200, num_seed_trees=5)


In [ ]:
leaf_map = gtree.calculate_leaf_map(result['tree'], features, targets, gtree.leaf_good_rate_split_builder)

print gtree.error_rate_loss(result['tree'].predict(features_validation, leaf_map), targets_validation)

In [ ]:
generations[-1]['best_of_generation']['tree'].find_leaves(features).value_counts()

In [ ]:
for gen in generations[-1]['generation']:
    print '--------------------{:.4f}----------------------------'.format(gen['loss_testing'])
    gen['tree'].prn()

In [ ]:
for result in generations[-1]['generation']:
    print '---------------------------------------------'
    result['tree'].prn()

In [ ]:
result = gtree.train_random_trees(features, targets, loss_fn=gtree.error_rate_loss,
                                  max_depth=2,
                                  min_to_split=10,
                                  num_trees=10)

In [ ]:
leaf_map = gtree.calculate_leaf_map(result['tree'], features, targets, gtree.leaf_good_rate_split_builder)

print gtree.error_rate_loss(result['tree'].predict(features_validation, leaf_map), targets_validation)

In [ ]:
from sklearn import tree
from sklearn.model_selection import train_test_split

clf = tree.DecisionTreeClassifier(max_depth=2)
clf = clf.fit(features, targets)

In [ ]:
predictions = pd.Series(clf.predict_proba(features_validation)[:, 1], index=features_validation.index)
gtree.error_rate_loss(predictions, targets_validation)

In [ ]:
from sklearn.externals.six import StringIO  
from sklearn import tree as sklearn_tree
import pydot 
dot_data = StringIO() 
sklearn_tree.export_graphviz(clf, out_file=dot_data) 
graph = pydot.graph_from_dot_data(dot_data.getvalue()) 
graph.write_pdf("iris.pdf") 

In [ ]:
%alias_magic t timeit

In [ ]:
sel = features[features['feature_3'] < 0].index

In [ ]:
sel

In [ ]:
%t features.loc[sel]

In [ ]:
%t df.reindex_axis(sel, copy=False)

# Evolve

In [ ]:
# BC Dataset

In [8]:
#bc_info = datasets.load_breast_cancer()

features, target = datasets.make_hastie_10_2(n_samples=5000)

features = pd.DataFrame(features, dtype=np.float32)
target = pd.Series([1.0 if t == 1.0 else 0.0 for t in target], dtype=np.float32).dropna()
features = features.loc[target.index]

In [9]:
target.value_counts(dropna=False)

0.0    2543
1.0    2457
dtype: int64

In [10]:
gtree.tree_logger.setLevel(logging.WARNING)

In [11]:
gtree.tree_logger.setLevel(logging.WARNING)

result, generations = gtree.evolve(features, target,
                                   loss='cross_entropy',
                                   leaf_prediction='logit',
                                   max_depth=3,
                                   min_to_split=10,
                                   num_generations=5,
                                   num_survivors=20,
                                   num_children=50,
                                   num_seed_trees=10)

06:15:29 evolution DEBUG:Growing Seed: 1 of 10
06:15:29 evolution DEBUG:Growing Seed: 2 of 10
06:15:29 evolution DEBUG:Growing Seed: 3 of 10
06:15:29 evolution DEBUG:Growing Seed: 4 of 10
06:15:30 evolution DEBUG:Growing Seed: 5 of 10
06:15:30 evolution DEBUG:Growing Seed: 6 of 10
06:15:30 evolution DEBUG:Growing Seed: 7 of 10
06:15:30 evolution DEBUG:Growing Seed: 8 of 10
06:15:31 evolution DEBUG:Growing Seed: 9 of 10
06:15:31 evolution DEBUG:Growing Seed: 10 of 10
06:15:31 evolution DEBUG:Resplitting the data
06:15:31 evolution DEBUG:Mating to create 50 children
06:15:39 evolution DEBUG:Calculating loss functions for generation of size: 36
06:15:42 evolution DEBUG:Surviving Generation: 0:3.9755, 1:3.9755, 1:3.9755, 1:4.0522, 1:4.0830, 1:4.1213, 1:4.1904, 1:4.1904, 1:4.2134, 1:4.2134, 1:4.2211, 0:4.2287, 0:4.2441, 1:4.2748, 1:4.2748, 0:4.2824, 1:4.2902, 1:4.3132, 0:4.3132, 1:4.3132
06:15:42 evolution INFO:Generation 0 Training Loss: 4.0128 Hold Out Loss 3.9755

06:15:42 evolution DEBU

In [12]:
generations

[{'best_of_generation': {'gen': 0,
   'leaf_map': {1710399003: <tree._my_tree.LogitMapper at 0x1109b4a80>,
    2537835901: <tree._my_tree.LogitMapper at 0x1109b45d0>,
    2645965834: <tree._my_tree.LogitMapper at 0x11090ed50>,
    2811015970: <tree._my_tree.LogitMapper at 0x1108e2d50>,
    2960185972: <tree._my_tree.MeanLeafMapper at 0x1108fd5d0>,
    3075435824: <tree._my_tree.LogitMapper at 0x1108e2c60>,
    3433636729: <tree._my_tree.LogitMapper at 0x1108e2a80>,
    3673457714: <tree._my_tree.LogitMapper at 0x1108e2e40>},
   'loss_testing': 3.990842819213867,
   'loss_training': 4.1349382400512695,
   'tree': <gtree.BranchNode at 0x1107a8310>},
  'generation': [{'gen': 0,
    'leaf_map': {1710399003: <tree._my_tree.LogitMapper at 0x1109b4a80>,
     2537835901: <tree._my_tree.LogitMapper at 0x1109b45d0>,
     2645965834: <tree._my_tree.LogitMapper at 0x11090ed50>,
     2811015970: <tree._my_tree.LogitMapper at 0x1108e2d50>,
     2960185972: <tree._my_tree.MeanLeafMapper at 0x1108fd5d

In [13]:
for gen in generations:
    best = gen['best_of_generation']
    print '=========================={:.4f} {:.4f}==============================\n'.format(
        best['loss_training'],
        best['loss_testing'])
    best['tree'].prn()

==========================4.1349 3.9908==============================

			<Leaf 2811015970>
		0 -0.15553
			<Leaf 3075435824>
	6 -1.82841
			<Leaf 2645965834>
		4 1.22969
			<Leaf 2537835901>
6 -1.68725
			<Leaf 2960185972>
		3 -2.15368
			<Leaf 3433636729>
	3 -1.49140
			<Leaf 3673457714>
		0 1.25438
			<Leaf 1710399003>
==========================4.0738 4.0062==============================

			<Leaf 3597938505>
		0 -0.15553
			<Leaf 2761014288>
	1 -0.89579
			<Leaf 2820397278>
		4 1.22969
			<Leaf 2589462510>
6 -1.68725
			<Leaf 2014374955>
		9 0.24094
			<Leaf 2966650312>
	3 -1.49140
			<Leaf 2065402284>
		0 1.25438
			<Leaf 3287211746>
==========================4.1302 4.0062==============================

			<Leaf 3297464837>
		1 0.40369
			<Leaf 2623157997>
	6 -1.82841
			<Leaf 2860431884>
		0 0.21779
			<Leaf 2679692350>
6 -1.68725
			<Leaf 2495911885>
		3 -2.15368
			<Leaf 2059949459>
	3 -1.49140
			<Leaf 2685307669>
		0 1.25438
			<Leaf 2278115293>
==========================3.92

In [ ]:
generations[-1]['best_of_generation']

In [ ]:
for k, v in generations[-1]['best_of_generation']['leaf_map'].iteritems():
    print k, v.get_coeficients(), '\n'

In [ ]:
generations[-1]['best_of_generation']['leaf_map']

In [ ]:
gtree.tree_logger.setLevel(logging.WARNING)

result, generations = gtree.train_random_trees(features, target,
                                   loss='cross_entropy',
                                   leaf_prediction='logit',
                                   max_depth=3,
                                   min_to_split=10)

In [ ]:
X = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])
X

In [ ]:
fns = {0: lambda x: (x[:,0] + x[:,1] + x[:,2]).reshape(len(x), 1), 1: lambda x: (-1*x[:,0]).reshape(len(x), 1)}

hashes = np.array([[0], [1], [0]])
hashes

In [ ]:
predictions = np.zeros((len(X), 1))
predictions

In [ ]:
zero = np.zeros((len(X), 1))

for i in [0, 1]:
    #comparison = np.full((len(X), 1), i)
    predictions[hashes.reshape(len(X))==i] = fns[i](X[hashes.reshape(len(X))==i, :]) # += np.where(hashes==i, fns[i](X), zero)
    
predictions

In [ ]:
X[hashes==comparison, :]

In [ ]:
X[:,0].reshape(3, 1)

In [ ]:
X[hashes==np.array([1]).reshape((len(X), 1))]

In [ ]:
hashes==1

In [ ]:
X[np.array([True, False, True]), :]

In [ ]:
import numpy as np
import statsmodels.discrete.discrete_model as sm
import statsmodels.tools.tools as sm_tools

X = np.array([[1, 2,  3],
              [2, 7,  5],
              [3, 10, 7],
              [5, 18, 10],
              [-10, 70, 3]             
             ], dtype=np.float64)

y = np.array([[1], [0], [1], [0], [1]], dtype=np.float64)

logit = sm.Logit(y, sm_tools.add_constant(X))
fit = logit.fit_regularized(method='l1', alpha=1.0)
fit.params

In [ ]:
logit.predict(fit.params, sm_tools.add_constant(X))

In [ ]:
from sklearn.svm.base import _fit_liblinear

coef_, intercept_, n_iter = _fit_liblinear(
                X, np.ravel(y), C=1.0, fit_intercept=True, intercept_scaling=1.0,
                class_weight=None, penalty='l1', dual=False, verbose=True,
                max_iter=5000, tol=1e-4, random_state=None,
                sample_weight=None)
#n_iter = np.array([n_iter])

(coef_, intercept_, n_iter)

In [ ]:
from scipy.special import expit
expit(coef_.dot(X.T) + intercept_)